# ----- Neste código irei validar a temperatua

a etapa de validação consiste em verificar se os dados de temperatura obtidos por sensores remotos tem correlação com os dados in situ. Para tanto os procedimentos devem ser seguidos:

- Importar os dados de temperatura in situ
- Importar os dados de temperatura obtidos por sensores remotos
- Verificar se os dados possuem distribuição normal
- aplicar teste de correlação adequado de acordo com a distribuição dos dados

# Bibliotecas

In [1]:
#!pip install datetime
from datetime import date

#Bibliotecas
#! pip install geopandas
import geopandas as gpd
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta 
# ------ Visualizar todos os arquivo da pasta
from os import chdir, getcwd, listdir
from os.path import isfile
#!pip install xlrd>=1.2.0
# importing re for regular expressions
import re

> *Data da última atualização*

In [2]:
today = date.today()
print("DATA DA ÚLTIMA ATUALIZAÇÃO: ", today)

DATA DA ÚLTIMA ATUALIZAÇÃO:  2023-06-07


# Funções

In [5]:
# -------- Join entre shape e df
def join_df_to_shp(df,ano, gdf,codigoDF,codigoGDF,geometriaGDF):

    # Prepara a coluna da precipitação
    #(bancoPrecipitacao[(bancoPrecipitacao['ANO'] ==ano)].groupby(by=['CODIGO','ANO']).sum().groupby(level=[0]).cumsum()/12).to_csv('/content/drive/My Drive/desenvolvimento/'+str(ano)+'.csv')
    #bancoPrecipitacaoColunas = pd.read_csv('/content/drive/My Drive/desenvolvimento/'+str(ano)+'.csv',sep=",")
    """"""

    df_ = pd.merge(df, gdf[[codigoGDF,geometriaGDF]], left_on=codigoDF, right_on=codigoGDF, how='left')

    # DROP the NI counties
    df_ = df_.dropna(subset=[geometriaGDF])

    # convert back to a geopandas object (for ease of plotting etc.)
    crs = {'init': 'epsg:4326'}
    gdf_ = gpd.GeoDataFrame(df_, crs=crs, geometry=geometriaGDF)
    # remove the extra area_code column joined from gdf
    gdf_.drop(codigoGDF,axis=1, inplace=True)

    return gdf_

# Carregando os dados de Temperatura Média (ºC) INMET

In [4]:
path = r"H:\Meu Drive\Mestrado\PESQUISA\DATASET\Inmet"
# --------- Base de dados de temperatura utilizando o TRMM como grid de base
#temperatura_BD = pd.read_csv(path+"/DATASET/BD_TemperaturaMédiaCompleto.csv")

# -------- Estações automáticas (INMET)
#temperaturaEstacaoAutomatica = pd.read_csv(path+"/DATASET/Inmet")
# -------- Estações convencionais (INMET)
#!pip install --upgrade --force-reinstall xlrd
chdir(path)
print(getcwd())

# --------- Criando uma lista para visualizar todos os arquivos
todosOsArquivos = []
for c in listdir():
    if isfile(c):
        todosOsArquivos.append(c)

# ---- criar um dataframe com a listagem de todos os arquivos
df_arquivosDiretorio = pd.DataFrame()

# ---- criando uma coluna com o nomes dos arquivos
df_arquivosDiretorio["todosOsArquivos"]=todosOsArquivos

# ------ Aplicando um filtro de acordo com o nome do arquivo
df_arquivosDiretorio = df_arquivosDiretorio[df_arquivosDiretorio["todosOsArquivos"].str.contains("dados")]

# ----- Visualizando todos os arquivos no diretorio
df_arquivosDiretorio

# ------ Criando um df vazio para colocar os dados iniciais da estação como Nome, Codigo Estacao, etc
dadosEstacao = pd.DataFrame()
# ------ Criando um df vazio para colocar os dados climaticos
dadosClimaticos = pd.DataFrame()


# ------ Criando um laço para ir lendo cada arquivo do diretorio
for i in range(len(df_arquivosDiretorio['todosOsArquivos'])):
    
    # ----- Concatenando todas as planilhas para os das estações
    dadosEstacao = pd.concat([ dadosEstacao,
                              pd.read_csv(path + "\\" + df_arquivosDiretorio['todosOsArquivos'][i], engine="python", sep=";", 
                                          nrows=9, 
                                          header = None)])


# ------ Fazendo a operação de split com o separador ': '     
dadosEstacao = dadosEstacao[0].str.split(": ", expand = True).reset_index(drop=True)

# ----- Criando um df vazio para ir colocando cada linha
df = pd.DataFrame()
for coluna in dadosEstacao[0].unique():
    df[coluna] = dadosEstacao[dadosEstacao[0]==coluna][1].reset_index(drop=True)
    
df_inmet = pd.DataFrame()
# ------ Criando um laço para ir lendo cada arquivo do diretorio
for i in range(len(df_arquivosDiretorio['todosOsArquivos'])):
    dadosClimaticos = pd.read_csv(path + "\\" + df_arquivosDiretorio['todosOsArquivos'][i], engine="python", sep=";",
                                             skiprows=10)
    
    dadosClimaticos = pd.concat([df.iloc[i:i+1],dadosClimaticos])
    df_inmet = pd.concat([df_inmet,dadosClimaticos])

# applying ffill() method to fill the missing values
df_inmet[df.columns] = df_inmet[df.columns].ffill(axis = 0)

df_inmet    

H:\Meu Drive\Mestrado\PESQUISA\DATASET\Inmet


,Nome,Codigo Estacao,Latitude,Longitude,Altitude,Situacao,Data Inicial,Data Final,Periodicidade da Medicao,Data Medicao,"NUMERO DE DIAS COM PRECIP. PLUV, MENSAL (AUT)(número)","PRECIPITACAO TOTAL, MENSAL (AUT)(mm)","PRESSAO ATMOSFERICA, MEDIA MENSAL (AUT)(mB)","TEMPERATURA MEDIA, MENSAL (AUT)(°C)","VENTO, VELOCIDADE MAXIMA MENSAL (AUT)(m/s)","VENTO, VELOCIDADE MEDIA MENSAL (AUT)(m/s)",Unnamed: 7,"TEMPERATURA MEDIA COMPENSADA, MENSAL(°C)",Unnamed: 2
0,ARCO VERDE,A309,-8.4336111,-37.05555555,683.95,Operante,2004-11-19,2020-12-31,Mensal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ARCO VERDE,A309,-8.4336111,-37.05555555,683.95,Operante,2004-11-19,2020-12-31,Mensal,2004-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARCO VERDE,A309,-8.4336111,-37.05555555,683.95,Operante,2004-11-19,2020-12-31,Mensal,2004-12-31,NaN,NaN,NaN,25.461334,8.8,NaN,NaN,NaN,NaN
2,ARCO VERDE,A309,-8.4336111,-37.05555555,683.95,Operante,2004-11-19,2020-12-31,Mensal,2005-01-31,6.0,28.2,935.009368,26.302273,8.5,3.238441,NaN,NaN,NaN
3,ARCO VERDE,A309,-8.4336111,-37.05555555,683.95,Operante,2004-11-19,2020-12-31,Mensal,2005-02-28,6.0,78.0,935.141175,25.893323,8.0,3.326048,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,PETROLINA,82983,-9.3886111,-40.52333332,372.54,Operante,1998-01-01,2020-12-31,Mensal,2020-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.060000,NaN
272,PETROLINA,82983,-9.3886111,-40.52333332,372.54,Operante,1998-01-01,2020-12-31,Mensal,2020-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.948667,NaN
273,PETROLINA,82983,-9.3886111,-40.52333332,372.54,Operante,1998-01-01,2020-12-31,Mensal,2020-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.671333,NaN
274,PETROLINA,82983,-9.3886111,-40.52333332,372.54,Operante,1998-01-01,2020-12-31,Mensal,2020-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.251034,NaN
